In [23]:
import sqlite3
import pickle
import pandas as pd

In [62]:
shutuba_table = pd.read_pickle("doyou_score")
shutuba_table.drop(columns=['斤量','jockey_id', 'trainer_id', '体重'\
                            , '体重変化', 'n_horses', 'course_len', 'weather' \
                            , 'race_type', 'ground_state', 'around'], inplace=True)

In [63]:
print(shutuba_table.columns)

Index(['枠番', '馬番', '単勝', 'horse_id', '性', '年齢', 'date', 'race_class', '開催',
       'start_time', 'oddsrank_score', 'rank1to1_score', 'speed_score'],
      dtype='object')


In [64]:
shutuba_table = shutuba_table.astype(str) # 全体を文字列に
shutuba_table.reset_index(inplace=True)
shutuba_table = shutuba_table.rename(columns={'index': 'race_id'}) # インデックスを列に

In [65]:
conn = sqlite3.connect('races.sqlite')
cursor = conn.cursor()

**********

runnersテーブルのodds

In [67]:
runner_table = shutuba_table[['race_id', '馬番', '枠番', '開催', '単勝', 'horse_id']].copy()
# runner_table = shutuba_table[['race_id', '馬番', '枠番', '騎手名', '単勝', 'horse_id']].copy()
runner_table.rename(columns={'馬番' : 'number', 
                             '枠番' : 'position',
                             '開催' : 'jockey',
                             '単勝' : 'odds'}, 
                  inplace=True)

In [87]:
cursor.execute('SELECT * FROM runners')
data = cursor.fetchall()

# データを表示
for row in data:
    print(row)

('202303020701', '1', '1', '03', '24.9', '2021106804')
('202303020701', '2', '2', '03', '112.7', '2021102302')
('202303020701', '3', '2', '03', '8.7', '2021105297')
('202303020701', '4', '3', '03', '214.5', '2021103343')
('202303020701', '5', '3', '03', '3.1', '2021106866')
('202303020701', '6', '4', '03', '10.5', '2021105060')
('202303020701', '7', '4', '03', '454.7', '2021107294')
('202303020701', '8', '5', '03', '51.7', '2021102391')
('202303020701', '9', '5', '03', '13.1', '2021101023')
('202303020701', '10', '6', '03', '239.1', '2021101152')
('202303020701', '11', '6', '03', '3.1', '2021105057')
('202303020701', '12', '7', '03', '9.6', '2021104127')
('202303020701', '13', '7', '03', '14.2', '2021100236')
('202303020701', '14', '8', '03', '81.6', '2021104618')
('202303020701', '15', '8', '03', '136.3', '2021100746')
('202303020702', '1', '1', '03', '3.1', '2021106963')
('202303020702', '2', '2', '03', '9.0', '2021104112')
('202303020702', '3', '3', '03', '19.4', '2021103128')
('202

In [84]:
ids = shutuba_table['race_id'].unique()
for i in range(ids.shape[0]):
    conn.execute("DELETE from runners where race_id=?;", (ids[i],))
conn.commit()

In [86]:
# 新しいオッズにするために、該当のrace_idのデータを一度すべて消して新しく書き込む
for i in range(ids.shape[0]):
    new_odds_table = runner_table[runner_table['race_id'] == ids[i]]
    new_odds_table.to_sql('runners', conn, if_exists='append', index=None)


*****************

predictsテーブル

In [89]:
pred_table = shutuba_table[['race_id', '馬番', 'oddsrank_score', 'rank1to1_score', 'speed_score', '開催']].copy()

In [90]:
pred_table.rename(columns={'馬番' : 'runners_number', 
                           'oddsrank_score' : 'model1',
                            'rank1to1_score' : 'model2',
                            'speed_score' : 'model3',
                        #   'モデル4' : 'model4'
                            '開催' : 'model4',}, 
                  inplace=True)

In [92]:
pred_table.to_sql('predicts', conn, if_exists='append', index=None)

476

*************

In [93]:
conn.close()